# Merge Erddap Cruise Data and Erddap Nut Data to generate a csv file 

- merged data will need to account for when nutrients arent at a ctd location

In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np


In [2]:
server_url = 'http://akutan.pmel.noaa.gov:8080/erddap'

In [6]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for=f'CTD final'))
#print(df['Dataset ID'].values)

cruises = list(set([x.split('_')[1] for x in df['Dataset ID']]))
#cruises = ['dy1907']

In [7]:
output_path = '/home/akutan/bell/in_and_outbox/erddap_generated/merged_ctdnutdata_from_erddap/'
f=open("erddap_ctdmerged_from_tabledap.xml", "a+")

for cruise in sorted(cruises):
    print(f'Searching for {cruise}')
    df = pd.read_csv(e.get_search_url(response='csv', search_for=f'CTD {cruise}'))

    dfs = {}

    for dataset_id in sorted(df['Dataset ID'].values):
        if ('preliminary' in dataset_id) or ('merged' in dataset_id):
            pass

        else:
            d = ERDDAP(server=server_url,
                protocol='tabledap',
                response='csv'
            )
            d.dataset_id=dataset_id

            df_m = d.to_pandas(
                        index_col='time (UTC)',
                        parse_dates=True,
                        skiprows=(1,)  # units information can be dropped.
                        )
            df_m.sort_index(inplace=True)
            df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

            dfs.update({dataset_id:df_m})

    if not ('nuts' in ",".join(sorted(df['Dataset ID'].values))):
        print(f'No nutrient data for {cruise}')
        pass
    else:
        try:
            dfs[f'CTD_{cruise}_final_nuts']['pressure'] = dfs[f'CTD_{cruise}_final_nuts']['pressure'].round()
            dfs[f'CTD_{cruise}_final']['time (UTC)'] = dfs[f'CTD_{cruise}_final'].index
            dfs[f'CTD_{cruise}_final_nuts']['time (UTC)'] = dfs[f'CTD_{cruise}_final_nuts'].index
            dfs[f'CTD_{cruise}_final']['profile_id'] = [x.split('_')[0] for x in dfs[f'CTD_{cruise}_final']['profile_id']]
            dfs[f'CTD_{cruise}_final_nuts']['profile_id'] = [x.split('_')[0] for x in dfs[f'CTD_{cruise}_final_nuts']['profile_id']]

            nut_df = pd.merge(
                dfs[f'CTD_{cruise}_final'],
                dfs[f'CTD_{cruise}_final_nuts'],
                how="left",
                on=["pressure","profile_id"],
            )
            nut_df.rename(columns={"latitude_x": "latitude",
                                   'longitude_x': 'longitude',
                                   'BTL_103_y': 'BTL_103',
                                   'time (UTC)_x': 'time',
                                   'Station_Name_x': 'Station_Name',
                                   'Water_Depth_x':'Water_Depth'}, inplace=True)
            nut_df['time_str']=nut_df["time"].dt.strftime("%Y-%m-%d %H:%M:%S")
            nut_df.drop(['time','time (UTC)_y','latitude_y','longitude_y',
                         'BTL_103_x','Water_Depth_y','Station_Name_y'],axis=1).to_csv(f'{cruise}_merged.csv')

            
        except:
            if not cruise in ['he1901']:
                continue
            dfs[f'CTD_{cruise}_final_nuts']['pressure'] = dfs[f'CTD_{cruise}_final_nuts']['pressure'].round()
            dfs[f'CTD_{cruise}']['time (UTC)'] = dfs[f'CTD_{cruise}'].index
            dfs[f'CTD_{cruise}_final_nuts']['time (UTC)'] = dfs[f'CTD_{cruise}_final_nuts'].index
            dfs[f'CTD_{cruise}']['profile_id'] = [x.split('_')[0] for x in dfs[f'CTD_{cruise}']['profile_id']]
            dfs[f'CTD_{cruise}_final_nuts']['profile_id'] = [x.split('_')[0] for x in dfs[f'CTD_{cruise}_final_nuts']['profile_id']]

            nut_df = pd.merge(
                dfs[f'CTD_{cruise}'],
                dfs[f'CTD_{cruise}_final_nuts'],
                how="left",
                on=["pressure","profile_id"],
            )
            nut_df.rename(columns={"latitude_x": "latitude",
                                   'longitude_x': 'longitude',
                                   'time (UTC)_x': 'time',}, inplace=True)
            nut_df['time_str']=nut_df["time"].dt.strftime("%Y-%m-%d %H:%M:%S")
            nut_df.drop(['time','time (UTC)_y','latitude_y','longitude_y'],axis=1).to_csv(f'{cruise}_merged.csv')

        #write out xml file
        f.write(f"""
        <dataset type="EDDTableFromAsciiFiles" datasetID="CTD_{cruise}_final_merged" active="true">
            <reloadEveryNMinutes>10080</reloadEveryNMinutes>
            <updateEveryNMillis>10000</updateEveryNMillis>
            <fileDir>{output_path}</fileDir>
            <fileNameRegex>{cruise}_merged.csv</fileNameRegex>
            <recursive>false</recursive>
            <pathRegex>.*</pathRegex>
            <metadataFrom>last</metadataFrom>
            <charset>ISO-8859-1</charset>
            <columnSeparator>,</columnSeparator>
            <columnNamesRow>1</columnNamesRow>
            <firstDataRow>2</firstDataRow>
            <preExtractRegex></preExtractRegex>
            <postExtractRegex></postExtractRegex>
            <extractRegex></extractRegex>
            <columnNameForExtract></columnNameForExtract>
            <sortedColumnSourceName>time_str</sortedColumnSourceName>
            <sortFilesBySourceNames>time_str</sortFilesBySourceNames>
            <fileTableInMemory>false</fileTableInMemory>
            <!-- sourceAttributes>
            </sourceAttributes -->
            <addAttributes>
            <att name="cdm_data_type">Profile</att>
            <att name="cdm_altitude_proxy">pressure</att> 
            <att name="cdm_profile_variables">profile_id</att>
            <att name="subsetVariables">profile_id, Station_Name</att>
            <att name="Conventions">COARDS, CF-1.6, ACDD-1.3</att>
            <att name="infoUrl">https://pmel.noaa.gov/</att>
            <att name="institution">NOAA/PMEL - EcoFOCI</att>
            <att name="keywords">merged, chlorophyll, concentration, concentration_of_chlorophyll_in_sea_water, conductivity, ctd, data, density, depth, depth2, dissolved, dissolved o2, kg/m**3, latitude, local, m**3, micromoles, micromoles/l, mole, O2, O_65, ocean, ocean color, oceans,
            Salinity/Density &gt; Salinity, OST_62, oxygen, percent, practical, profile, psu, S_41, S_42, salinity, sat, sea, sea_water_practical_salinity, seawater, secondary, sigma, sigma-t, silicate, sonde, source, ST_70, profile_id, statistics, T2_35, T_28, temperature, time, Tr_904, transmissometry, umol, umol/kg, unit, water</att>
            <att name="keywords_vocabulary">GCMD Science Keywords</att>
            <att name="license">[standard]</att>
            <att name="sourceUrl">(local files)</att>
            <att name="standard_name_vocabulary">CF Standard Name Table v55</att>
            <att name="summary">CTD Data from {cruise.upper()} - Final Merged </att>
            <att name="title">CTD Data from {cruise.upper()} - Final Merged </att>
            </addAttributes>
            <dataVariable>
                <sourceName>time_str</sourceName>
                <destinationName>time</destinationName>
                <dataType>String</dataType>
                <!-- sourceAttributes>
                </sourceAttributes -->
                <addAttributes>
                    <att name="ioos_category">Time</att>
                    <att name="long_name">time</att>
                    <att name="source_name">time_str</att>
                    <att name="standard_name">time</att>
                    <att name="time_precision">1970-01-01T00:00:00Z</att>
                    <att name="units">yyyy-MM-dd HH:mm:ss</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>pressure</sourceName>
                <destinationName>pressure</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">1</att>
                    <att name="type">EVEN</att>
                    <att name="units">dbar</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">8000.0</att>
                    <att name="colorBarMinimum" type="double">-8000.0</att>
                    <att name="colorBarPalette">TopographyDepth</att>
                    <att name="ioos_category">Pressure</att>
                    <att name="long_name">Pressure</att>
                    <att name="standard_name">sea_water_pressure</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>latitude</sourceName>
                <destinationName>latitude</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">500</att>
                    <att name="type">EVEN</att>
                    <att name="units">degree_north</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">90.0</att>
                    <att name="colorBarMinimum" type="double">-90.0</att>
                    <att name="ioos_category">Location</att>
                    <att name="long_name">Latitude</att>
                    <att name="standard_name">latitude</att>
                    <att name="units">degrees_north</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>longitude</sourceName>
                <destinationName>longitude</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">501</att>
                    <att name="type">EVEN</att>
                    <att name="units">degree_west</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="ioos_category">Location</att>
                    <att name="long_name">Longitude</att>
                    <att name="standard_name">longitude</att>
                    <att name="units">degrees_east</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>BTL_103</sourceName>
                <destinationName>BTL_103</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">103</att>
                    <att name="generic_name">BTL</att>
                    <att name="long_name">NISKIN BOTTLE NUMBER</att>
                    <att name="name">BTL_103</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">100.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Statistics</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>S_42</sourceName>
                <destinationName>S_42</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">42</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">sal</att>
                    <att name="long_name">SALINITY (PSU)</att>
                    <att name="name">S_42</att>
                    <att name="units">psu</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">37.0</att>
                    <att name="colorBarMinimum" type="double">32.0</att>
                    <att name="ioos_category">Salinity</att>
                    <att name="standard_name">sea_water_practical_salinity</att>
                    <att name="units">PSU</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>S_41</sourceName>
                <destinationName>S_41</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">41</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">sal</att>
                    <att name="long_name">SALINITY (PSU)</att>
                    <att name="name">S_41</att>
                    <att name="units">psu</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">37.0</att>
                    <att name="colorBarMinimum" type="double">32.0</att>
                    <att name="ioos_category">Salinity</att>
                    <att name="standard_name">sea_water_practical_salinity</att>
                    <att name="units">PSU</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>ST_70</sourceName>
                <destinationName>ST_70</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">70</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="FORTRAN_format">f10.2</att>
                    <att name="generic_name">den</att>
                    <att name="long_name">SIGMA-T (KG/M**3)</att>
                    <att name="name">ST_70</att>
                    <att name="units">kg m-3</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">200.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Unknown</att>
                    <att name="standard_name">sea_water_sigma_t</att>
                    <att name="units">kg m-3</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>Fch_906</sourceName>
                <destinationName>Fch_906</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">906</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="long_name">Chlorophyll A</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="epic_key">Fch_906</att>
                    <att name="units">ugrams/l</att>
                    <att name="_FillValue">1.0E35</att>
                    <att name="colorBarMaximum" type="double">30.0</att>
                    <att name="colorBarMinimum" type="double">0.03</att>
                    <att name="colorBarScale">Log</att>
                    <att name="ioos_category">Ocean Color</att>
                    <att name="standard_name">concentration_of_chlorophyll_in_sea_water</att>
                    <att name="units">ugrams/l</att>        </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>T_28</sourceName>
                <destinationName>T_28</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">28</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="FORTRAN_format">f10.2</att>
                    <att name="generic_name">temp</att>
                    <att name="long_name">TEMPERATURE (C)</att>
                    <att name="name">T_28</att>
                    <att name="units">degree_C</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="ioos_category">Temperature</att>
                    <att name="standard_name">sea_water_temperature</att>
                    <att name="units">degree_C</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>OST_62</sourceName>
                <destinationName>OST_62</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">62</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">ox</att>
                    <att name="long_name">OXYGEN, &#37;SAT</att>
                    <att name="name">OST_62</att>
                    <att name="units">&#37;</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">100.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Dissolved O2</att>
                    <att name="units">percent</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>O_65</sourceName>
                <destinationName>O_65</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">65</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="FORTRAN_format">f10.2</att>
                    <att name="generic_name">ox</att>
                    <att name="long_name">OXYGEN (UMOL/KG)</att>
                    <att name="name">O_65</att>
                    <att name="units">umol/kg</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="ioos_category">Dissolved O2</att>
                    <att name="standard_name">moles_of_oxygen_per_unit_mass_in_sea_water</att>
                    <att name="units">umol/kg</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>T2_35</sourceName>
                <destinationName>T2_35</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">35</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="FORTRAN_format">f10.2</att>
                    <att name="generic_name">temp</att>
                    <att name="long_name">Secondary Temperature</att>
                    <att name="name">T2_35</att>
                    <att name="units">degree_C</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="ioos_category">Temperature</att>
                    <att name="standard_name">sea_water_temperature</att>
                    <att name="units">degree_C</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>Tr_904</sourceName>
                <destinationName>Tr_904</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">904</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="long_name">Transmissometry (CTD)</att>
                    <att name="name">Tr_904</att>
                    <att name="units">&#37;</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">100.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Unknown</att>
                    <att name="units">percent</att>
                    <att name="standard_name">seawater_optical_transmission_ratio</att>
                </addAttributes>
            </dataVariable>    
            <dataVariable>
                <sourceName>Water_Depth</sourceName>
                <destinationName>Water_Depth</destinationName>
                <dataType>float</dataType>
                <addAttributes>
                    <att name="missing_value" type="float">9999.0</att>
                    <att name="_FillValue" type="float">1.0e35</att>
                    <att name="ioos_category">Pressure</att>
                    <att name="standard_name">seafloor_depth</att>
                    <att name="units">dbar</att>
                    <att name="description">estimated seafloor depth at station by ek60 during cast</att>
                </addAttributes>
            </dataVariable>  
            <dataVariable>
                <sourceName>air_pressure_at_sealevel</sourceName>
                <destinationName>air_pressure_at_sealevel</destinationName>
                <dataType>float</dataType>
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                    <att name="_FillValue">1.0E35</att>
                    <att name="colorBarMaximum" type="double">1050.0</att>
                    <att name="colorBarMinimum" type="double">950.0</att>
                    <att name="ioos_category">Pressure</att>
                    <att name="name">air_pressure_at_sealevel</att>
                    <att name="units">mbar</att>
                </addAttributes>
            </dataVariable>  
            <dataVariable>
                <sourceName>wind_direction</sourceName>
                <destinationName>wind_direction</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-999.0</att>
                    <att name="_FillValue">1.0E35</att>
                    <att name="colorBarMaximum" type="double">360.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Wind</att>
                    <att name="long_name">Wind From Direction</att>
                    <att name="standard_name">wind_from_direction</att>
                    <att name="units">degrees</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>wind_speed</sourceName>
                <destinationName>wind_speed</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-99.9</att>
                    <att name="_FillValue">1.0E35</att>
                    <att name="ioos_category">Unknown</att>
                    <att name="long_name">wind speed</att>
                    <att name="units">m s-1</att>
                    <att name="standard_name">wind_speed</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>air_temperature</sourceName>
                <destinationName>air_temperature</destinationName>
                <dataType>float</dataType>
                <addAttributes>
                   <att name="missing_value" type="float">-99.9</att>
                    <att name="_FillValue">1.0E35</att>
                    <att name="colorBarMaximum" type="double">40.0</att>
                    <att name="colorBarMinimum" type="double">-10.0</att>
                    <att name="ioos_category">Temperature</att>
                    <att name="standard_name">air_temperature</att>
                    <att name="units">degree_C</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>Station_Name</sourceName>
                <destinationName>Station_Name</destinationName>
                <dataType>String</dataType>
                <addAttributes>
                    <att name="ioos_category">Unknown</att>
                    <att name="standard_name">foci_station_name</att>
                    <att name="description">FOCI station name</att>
                </addAttributes>
            </dataVariable>  
            <dataVariable>
                <sourceName>profile_id</sourceName>
                <destinationName>profile_id</destinationName>
                <dataType>String</dataType>
                <!-- sourceAttributes>
                    <att name="cf_role">profile_id</att>
                    <att name="long_name">timeseries id</att>
                </sourceAttributes -->
                <addAttributes>
                    <att name="ioos_category">Identifier</att>
                    <att name="cf_role">profile_id</att>
                </addAttributes>
            </dataVariable>       
            <dataVariable>
                <sourceName>SI_188</sourceName>
                <destinationName>SI_188</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">188</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">SI</att>
                    <att name="long_name">SILICATE (micromoles/l)</att>
                    <att name="name">SI_188</att>
                    <att name="units">uM/l</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">50.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Dissolved Nutrients</att>
                    <att name="standard_name">mole_concentration_of_silicate_in_sea_water</att>
                    <att name="units">uM/l</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>PO4_186</sourceName>
                <destinationName>PO4_186</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">186</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">PO4</att>
                    <att name="long_name">PHOSPHATE (micromoles/l)</att>
                    <att name="name">PO4_186</att>
                    <att name="units">uM/l</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">4.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Dissolved Nutrients</att>
                    <att name="standard_name">mole_concentration_of_phosphate_in_sea_water</att>
                    <att name="units">uM/l</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>NO2_184</sourceName>
                <destinationName>NO2_184</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">184</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">NO2</att>
                    <att name="long_name">NITRITE (micromoles/l)</att>
                    <att name="name">NO2_184</att>
                    <att name="units">uM/l</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">1.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Dissolved Nutrients</att>
                    <att name="standard_name">mole_concentration_of_nitrite_in_sea_water</att>
                    <att name="units">uM/l</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>NO3_182</sourceName>
                <destinationName>NO3_182</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">182</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">NO3</att>
                    <att name="long_name">NITRATE (micromoles/l)</att>
                    <att name="name">NO3_182</att>
                    <att name="units">uM/l</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">50.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Dissolved Nutrients</att>
                    <att name="standard_name">mole_concentration_of_nitrate_in_sea_water</att>
                    <att name="units">uM/l</att>
                </addAttributes>
            </dataVariable>
            <dataVariable>
                <sourceName>NH4_189</sourceName>
                <destinationName>NH4_189</destinationName>
                <dataType>float</dataType>
                <!-- sourceAttributes>
                    <att name="epic_code" type="int">182</att>
                    <att name="fill_value" type="double">1.0E35</att>
                    <att name="generic_name">NO3</att>
                    <att name="long_name">NITRATE (micromoles/l)</att>
                    <att name="name">NH4_189</att>
                    <att name="units">uM/l</att>
                </sourceAttributes -->
                <addAttributes>
                   <att name="missing_value" type="float">-9999.0</att>
                   <att name="_FillValue" type="float">1.0e35</att>
                    <att name="colorBarMaximum" type="double">50.0</att>
                    <att name="colorBarMinimum" type="double">0.0</att>
                    <att name="ioos_category">Dissolved Nutrients</att>
                    <att name="standard_name">mole_concentration_of_ammonium_in_sea_water</att>
                    <att name="units">uM/l</att>
                </addAttributes>
            </dataVariable>
        </dataset>
        """)

Searching for ae1001
No nutrient data for ae1001
Searching for ae1401
No nutrient data for ae1401
Searching for ae1402
No nutrient data for ae1402
Searching for ae1501
No nutrient data for ae1501
Searching for aq1201
No nutrient data for aq1201
Searching for aq1301
No nutrient data for aq1301
Searching for aq1401l2
No nutrient data for aq1401l2
Searching for aq1401l3
No nutrient data for aq1401l3
Searching for aq1501
No nutrient data for aq1501
Searching for aq1601
Searching for aq1801
No nutrient data for aq1801
Searching for be1201
No nutrient data for be1201
Searching for be1301
No nutrient data for be1301
Searching for be1302
No nutrient data for be1302
Searching for be1303
No nutrient data for be1303
Searching for cf1601
No nutrient data for cf1601
Searching for dy1006
No nutrient data for dy1006
Searching for dy1104
No nutrient data for dy1104
Searching for dy1207
No nutrient data for dy1207
Searching for dy1208
No nutrient data for dy1208
Searching for dy1408l1
No nutrient data 

/Users/bell/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


No nutrient data for hx196
Searching for hx200
No nutrient data for hx200
Searching for hx209
No nutrient data for hx209
Searching for hx213
No nutrient data for hx213
Searching for hx220
No nutrient data for hx220
Searching for hx222
No nutrient data for hx222
Searching for mb1101
No nutrient data for mb1101
Searching for me0823
No nutrient data for me0823
Searching for mf0610l2
No nutrient data for mf0610l2
Searching for mf0702
No nutrient data for mf0702
Searching for mf0706
No nutrient data for mf0706
Searching for mf0707
No nutrient data for mf0707
Searching for mf0708
No nutrient data for mf0708
Searching for mf0711l2
No nutrient data for mf0711l2
Searching for mf0802l1
No nutrient data for mf0802l1
Searching for mf0802l2
No nutrient data for mf0802l2
Searching for mf0810l2
No nutrient data for mf0810l2
Searching for mf0904l2
No nutrient data for mf0904l2
Searching for mf1006
No nutrient data for mf1006
Searching for nw1601
Searching for nw1602
Searching for nw1603
Searching for 